<a href="https://colab.research.google.com/github/AUT-Student/CN-Project/blob/main/CN_Project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import torch
import scipy
import scipy.sparse as sp
from torch.nn import Linear, Softmax
import numpy as np
import torch.nn.functional as F

In [2]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 20.2 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=4713926b8264d237fe034f9432ff97f0d3055c8b21429c3aa62f54022185a5fc
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.9/873.9 KB 55.6 MB/s eta 0:00:00


In [3]:
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix

Source:
1.   https://docs.dgl.ai/en/0.8.x/_modules/dgl/nn/pytorch/conv/sgconv.html
2.   https://github.com/Tiiiger/SGC

# Dataset Amazon

In [4]:
from torch_geometric.datasets.amazon import Amazon

In [6]:
amazon = Amazon("/content", "Photo")

Processing...
Done!


In [8]:
amazon[0]

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

# Dataset DBLP

### Load

In [4]:
from torch_geometric.datasets.dblp import DBLP

In [5]:
dblp = DBLP("/content")

Extracting /content/raw/DBLP_processed.zip
Processing...
Done!


### Overview

In [ ]:
# Source: https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/datasets/dblp.py

In [ ]:
dblp.data

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)

In [ ]:
dblp.num_edge_features

{('author', 'to', 'paper'): 0,
 ('paper', 'to', 'author'): 0,
 ('paper', 'to', 'term'): 0,
 ('paper', 'to', 'conference'): 0,
 ('term', 'to', 'paper'): 0,
 ('conference', 'to', 'paper'): 0}

In [ ]:
dblp.num_features

{'author': 334, 'paper': 4231, 'term': 50, 'conference': 0}

In [ ]:
dblp.data['author']

{'x': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'y': tensor([2, 2, 3,  ..., 0, 0, 0]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ...,  True,  True,  True])}

In [ ]:
dblp.data['paper']

{'x': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}

In [ ]:
set(dblp.data['author']["y"].numpy())

{0, 1, 2, 3}

In [ ]:
(sum(dblp.data['author']["train_mask"].numpy()), sum(dblp.data['author']["test_mask"].numpy()), sum(dblp.data['author']["val_mask"].numpy()))

(400, 3257, 400)

In [ ]:
dblp.num_features

{'author': 334, 'paper': 4231, 'term': 50, 'conference': 0, 'y': 0}

In [ ]:
dblp.data['author'].num_features

334

### Create Needed Features

In [ ]:
author_x = dblp.data["author"].x

In [ ]:
pta_edges = dblp[0].edge_index_dict[('paper','to','author')]
paper_auther_data = pd.DataFrame(pta_edges.numpy().T, columns=["paper", "author"])

In [ ]:
author_edges = []
for authors in paper_auther_data.groupby("paper")["author"].apply(np.array):
  for author1 in authors:
    for author2 in authors:
      if author1==author2:continue
      author_edges.append([author1, author2])

author_edges = torch.Tensor(np.array(author_edges).T).to(torch.long)

# Dataset

In [4]:
planetoid = Planetoid("/content/planetoid", name="PubMed")

Processing...
Done!


In [5]:
dataset = planetoid[0]

In [6]:
dataset

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [7]:
(sum(dataset.train_mask), sum(dataset.val_mask), sum(dataset.test_mask))

(tensor(60), tensor(500), tensor(1000))

# Train and Test

# Model

In [30]:
SkX.shape

torch.Size([19717, 500])

In [69]:
class SGC(torch.nn.Module):
    def __init__(self, number_features, number_classes, k):
      super().__init__()

      self.linear = Linear(number_features, number_classes)
      self.softmax = Softmax(dim=1)
      self.k = k

    def create_sgc_matrix(self, dataset):
      A = to_scipy_sparse_matrix(dataset.edge_index)
      I = sp.eye(dataset.num_nodes)

      A_tilde = A + I

      D_tilde = sp.diags(A_tilde.sum(axis=0).A1)

      S = D_tilde.power(-1/2) * A * D_tilde.power(-1/2)

      Sk = S.power(self.k)

      X = sp.csr_matrix(dataset.x)

      SkX = Sk * sp.csr_matrix(dataset.x)

      SkX = torch.tensor(SkX.todense()).to(torch.float32)

      return SkX

    def forward(self, x):
      x = self.linear(x)
      x = self.softmax(x)
      return x

In [70]:
model = SGC(number_features=500, number_classes=3, k=3)

In [71]:
def train(model, dataset):
  x = model.create_sgc_matrix(dataset)
  
  model.train()

  optimizer = torch.optim.Adam(model.parameters(), lr=0.2,
                           weight_decay=5e-6)
  
  for epoch in range(10000):
    pred = model(x)
    train_loss = F.cross_entropy(pred[dataset.train_mask],
                                  dataset.y[dataset.train_mask])
    
    validation_loss = F.cross_entropy(pred[dataset.val_mask],
                                      dataset.y[dataset.val_mask])

    print(f"epoch = {epoch} train_loss = {train_loss}, validation_loss={validation_loss}")

    # Backpropagation
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()  

In [72]:
train(model, dataset)

Streaming output truncated to the last 5000 lines.
epoch = 5000 train_loss = 1.0756762027740479, validation_loss=1.0954266786575317
epoch = 5001 train_loss = 1.0756762027740479, validation_loss=1.0954712629318237
epoch = 5002 train_loss = 1.0756759643554688, validation_loss=1.095457673072815
epoch = 5003 train_loss = 1.0756763219833374, validation_loss=1.0954447984695435
epoch = 5004 train_loss = 1.0756758451461792, validation_loss=1.0954749584197998
epoch = 5005 train_loss = 1.075676441192627, validation_loss=1.0954389572143555
epoch = 5006 train_loss = 1.0756758451461792, validation_loss=1.0954697132110596
epoch = 5007 train_loss = 1.0756762027740479, validation_loss=1.0954532623291016
epoch = 5008 train_loss = 1.0756759643554688, validation_loss=1.0954500436782837
epoch = 5009 train_loss = 1.0756759643554688, validation_loss=1.0954736471176147
epoch = 5010 train_loss = 1.0756762027740479, validation_loss=1.0954326391220093
epoch = 5011 train_loss = 1.0756759643554688, validation_los

In [64]:
def test(model, dataset):
  x = model.create_sgc_matrix(dataset)
  model.eval()
  with torch.no_grad():
    pred = model(x).argmax(dim=-1)

  accs = []
  for mask in [dataset.train_mask, dataset.val_mask, dataset.test_mask]:
      accs.append(int((pred[mask] == dataset.y[mask]).sum()) / int(mask.sum()))

  print(f"Train Accuracy = {accs[0]}")
  print(f"Valid Accuracy = {accs[1]}")
  print(f"Test  Accuracy = {accs[2]}")  

In [65]:
test(model, dataset)

Train Accuracy = 0.9166666666666666
Valid Accuracy = 0.684
Test  Accuracy = 0.655


In [ ]:
train(model)